In [1]:
# ps52
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=af797630af42c3baa9a0b6d52cb4f2de0b3048681ae794a2f1f023018fbf6eeb
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


🌟 𝐏𝐘𝐒𝐏𝐀𝐑𝐊 𝐏𝐫𝐚𝐜𝐭𝐢𝐜𝐞/𝐈𝐧𝐭𝐞𝐫𝐯𝐢𝐞𝐰 𝐏𝐫𝐨𝐛𝐥𝐞𝐦 📊
========================================
𝐐𝐮𝐞𝐫𝐢𝐞𝐬 𝐐𝐮𝐚𝐥𝐢𝐭𝐲 𝐀𝐧𝐝 𝐏𝐞𝐫𝐜𝐞𝐧𝐭𝐚𝐠𝐞 𝐏𝐫𝐨𝐛𝐥𝐞𝐦

We define query quality as: The average of the ratio between query rating and its position.
We also define poor query percentage as: The percentage of all queries with rating less than 3.
Write an pyspark code to find each query_name, the quality and poor_query_percentage.


𝐒𝐜𝐡𝐞𝐦𝐚 𝐀𝐧𝐝 𝐃𝐚𝐭𝐚:
================
 # Define the schema for the DataFrame
schema = StructType([
 StructField("query_name", StringType(), True),
 StructField("result", StringType(), True),
 StructField("position", IntegerType(), True),
 StructField("rating", IntegerType(), True)
])
# Create the data
data = [
 ("Dog", "Golden Retriever", 1, 5),
 ("Dog", "German Shepherd", 2, 5),
 ("Dog", "Mule", 200, 1),
 ("Cat", "Shirazi", 5, 2),
 ("Cat", "Siamese", 3, 3),
 ("Cat", "Sphynx", 7, 4)
]

In [2]:
# import all necessary module
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
# create spark session
spark = SparkSession.builder.master('local').appName('pyspark-day-5').getOrCreate()

In [4]:
spark

In [5]:
# define schema and data for df
schema = StructType([
 StructField("query_name", StringType(), True),
 StructField("result", StringType(), True),
 StructField("position", IntegerType(), True),
 StructField("rating", IntegerType(), True)
])
# Create the data
data = [
 ("Dog", "Golden Retriever", 1, 5),
 ("Dog", "German Shepherd", 2, 5),
 ("Dog", "Mule", 200, 1),
 ("Cat", "Shirazi", 5, 2),
 ("Cat", "Siamese", 3, 3),
 ("Cat", "Sphynx", 7, 4)
]

In [7]:
# create df for above
query = spark.createDataFrame(schema = schema, data = data)
query.show()

+----------+----------------+--------+------+
|query_name|          result|position|rating|
+----------+----------------+--------+------+
|       Dog|Golden Retriever|       1|     5|
|       Dog| German Shepherd|       2|     5|
|       Dog|            Mule|     200|     1|
|       Cat|         Shirazi|       5|     2|
|       Cat|         Siamese|       3|     3|
|       Cat|          Sphynx|       7|     4|
+----------+----------------+--------+------+



In [14]:
query.printSchema()

root
 |-- query_name: string (nullable = true)
 |-- result: string (nullable = true)
 |-- position: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [16]:
# make quality col then take avg
quality_df = query.withColumn('quality', col('rating')/col('position'))
quality_df.show()

+----------+----------------+--------+------+------------------+
|query_name|          result|position|rating|           quality|
+----------+----------------+--------+------+------------------+
|       Dog|Golden Retriever|       1|     5|               5.0|
|       Dog| German Shepherd|       2|     5|               2.5|
|       Dog|            Mule|     200|     1|             0.005|
|       Cat|         Shirazi|       5|     2|               0.4|
|       Cat|         Siamese|       3|     3|               1.0|
|       Cat|          Sphynx|       7|     4|0.5714285714285714|
+----------+----------------+--------+------+------------------+



In [17]:
# take avg
avg_quality_df = quality_df.groupBy('query_name').agg(avg('quality').alias('quality'))
avg_quality_df.show()

+----------+------------------+
|query_name|           quality|
+----------+------------------+
|       Cat|0.6571428571428571|
|       Dog|2.5016666666666665|
+----------+------------------+



In [18]:
# similarly make poor_query
poor_df = query.withColumn('poor_query',when(col('rating') < 3, 1).otherwise(0))
poor_df.show()

+----------+----------------+--------+------+----------+
|query_name|          result|position|rating|poor_query|
+----------+----------------+--------+------+----------+
|       Dog|Golden Retriever|       1|     5|         0|
|       Dog| German Shepherd|       2|     5|         0|
|       Dog|            Mule|     200|     1|         1|
|       Cat|         Shirazi|       5|     2|         1|
|       Cat|         Siamese|       3|     3|         0|
|       Cat|          Sphynx|       7|     4|         0|
+----------+----------------+--------+------+----------+



In [23]:
# cal percentage
poor_query_percentage = poor_df.groupBy('query_name').agg((sum('poor_query')/count('*')*100).alias('poor_query_percentage'))
poor_query_percentage.show()

+----------+---------------------+
|query_name|poor_query_percentage|
+----------+---------------------+
|       Cat|    33.33333333333333|
|       Dog|    33.33333333333333|
+----------+---------------------+



In [24]:
avg_quality_df.join(poor_query_percentage, on='query_name', how='inner').show()

+----------+------------------+---------------------+
|query_name|           quality|poor_query_percentage|
+----------+------------------+---------------------+
|       Cat|0.6571428571428571|    33.33333333333333|
|       Dog|2.5016666666666665|    33.33333333333333|
+----------+------------------+---------------------+

